# 필요한 패키지 불러오기

In [ ]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm 
import pickle
from numpy import dot
from numpy.linalg import norm
from itertools import chain
import statistics as stats
import gc
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.stats import pearsonr
# Fasttext 모델 임포트
from gensim.models import fasttext

ft_model = fasttext.load_facebook_model("C:\관계형테이블임베딩\cc.en.300.bin")

# 실험에 사용할 데이터 불러오기 및 전처리

In [ ]:
# 실험에 사용할 데이터 불러오기

file_list = os.listdir('C:\관계형테이블임베딩\데이터셋_20240520_N-Join-Pair')


In [ ]:
pop_file_list = []
data = []
for i in tqdm(range(len(file_list))):
    try:
        table = pd.read_csv('C:\관계형테이블임베딩\데이터셋_20240520_N-Join-Pair/{}'.format(file_list[i]), encoding = 'ISO-8859-1')
    except:
        pop_file_list.append(i)
        print(i)

In [ ]:
pop_file_list.sort(reverse=True)
for i in pop_file_list:
    file_list.pop(i)

In [ ]:
data = []
for i in range(len(file_list)):
    data.append(pd.read_csv('C:\관계형테이블임베딩\데이터셋_20240520_N-Join-Pair/{}'.format(file_list[i]), encoding = 'ISO-8859-1'))

In [ ]:
print(len(file_list), len(data))

In [ ]:
pop_list=[]
for i in range(len(data)):
    cat_columns = data[i].select_dtypes(exclude=np.number).columns.tolist()
    data[i] = data[i][cat_columns]
    data[i] = data[i].applymap(str)
    if (data[i].shape[0]) == 0 or (data[i].shape[1]) == 0:
        pop_list.append(i)
pop_list.sort(reverse=True)

for i in pop_list:
    data.pop(i)
    file_list.pop(i)

In [ ]:
for i in range(len(file_list)):
    print(i, " ", file_list[i])

In [ ]:
with open("./result_vector/table2vecFastText_file_list.pkl","wb") as f:
    pickle.dump(file_list, f)

# N-Join-Pair 코드

In [ ]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

def word_extract(tab):
    column_heading = tab.columns.tolist()
    column_value = tab.values.tolist()
    column_value = list(chain(*column_value))
    return list(set(column_heading + column_value))

def late_avg(list1, list2):
    late = 0
    for vec1 in list1:
        for vec2 in list2:
            late += cos_sim(vec1,vec2)
    if len(list1) * len(list2) > 0 :
        late_avg = late / (len(list1) * len(list2))
    else:
        return 0
    return late_avg

In [ ]:
tab_list = []
#tab_word_list = []
for i in tqdm(range(len(file_list))):
    tab_list.append(file_list[i])
    #tab_word_list.append(word_extract(data[i]))

In [ ]:
tab_wordvec_list= []
for word_list in tab_word_list:
    temp_list = []
    for word in word_list:
        try :
            temp_list.append(ft_model.wv[word])
        except:
            pass
    tab_wordvec_list.append(temp_list)

In [ ]:
tab_dict = {}
for i in range(len(tab_list)):
        tab_dict[tab_list[i]] = tab_wordvec_list[i]

In [ ]:
for i in tqdm(range(len(file_list))):
    if "@" in file_list[i] :
        print(file_list[i])
        
        a,b,c = file_list[i].split(" @ ")
        a,b,c = a.split("^")[0], b.split("^")[0], c.split("^")[0]
        print(f'조인된 테이블과 {a} 테이블간의 코사인유사도 : {late_avg(tab_dict[file_list[i]], tab_dict[a])} \n조인된 테이블과 {b} 테이블간의 코사인유사도 : {late_avg(tab_dict[file_list[i]], tab_dict[b])} \n조인된 테이블과 {c} 테이블간의 코사인유사도 : {late_avg(tab_dict[file_list[i]], tab_dict[c])} \n')

In [ ]:
len(tab_dict)

In [ ]:
with open("./result_vector/table2vecFastText.pkl","rb") as f:
    tab_wordvec_list = pickle.load(f)

In [ ]:
def normalize(lst):
    min_val = min(lst)
    max_val = max(lst)
    range_val = max_val - min_val
    if range_val == 0:  # 모든 값이 동일한 경우, 모든 값을 0으로 설정
        return [0 for _ in lst]
    else:
        return [(item - min_val) / range_val for item in lst]

def standardize(lst):
    mean = stats.mean(lst)
    stdev = stats.stdev(lst)
    return [(i - mean) / stdev for i in lst]

In [ ]:
def check_values(a, b, c):
    invalid_values = {"airlines.csv", "List of Unicorns in the World.csv", "loan-train.csv"}
    if a in invalid_values or b in invalid_values or c in invalid_values:
        return False
    return True

In [ ]:
cos_list = []
join_tab_list = []
cos_dict = {}
for i in tqdm(range(len(file_list))):
    if "@" in file_list[i] :
        a,b,c = file_list[i].split(" @ ")
        a,b,c = a.split("^")[0], b.split("^")[0], c.split("^")[0]
        if check_values(a,b,c):
            join_tab_list.append(file_list[i])
            #cos_list.append(late_avg(tab_dict[file_list[i]], tab_dict[a]) + late_avg(tab_dict[file_list[i]], tab_dict[b]) + late_avg(tab_dict[file_list[i]], tab_dict[c]))

In [ ]:
with open("./result_vector/table2vecFastText_coslist.pkl","rb") as f:
    cos_list = pickle.load(f)

In [ ]:
cos_list = standardize(cos_list)

In [ ]:
pop_temp = []
for i in range(len(cos_list)):
    if cos_list[i]>2 or cos_list[i]<-2:
        pop_temp.append(i)
pop_temp.sort(reverse=True)
print(pop_temp)
for i in pop_temp:
    join_tab_list.pop(i)
    cos_list.pop(i)

In [ ]:
cos_list = normalize(cos_list)

In [ ]:
for i in range(len(join_tab_list)):
        
        cos_dict[join_tab_list[i]] = cos_list[i]


In [ ]:
cos_dict

# TF-IDF 계산 및 TF-IDF overlapping을 활용한 N-Join-Pair 성능평가

In [ ]:
def word_extract_value(tab):
    #column_heading = tab.columns.tolist()
    column_value = tab.values.tolist()
    column_value = list(chain(*column_value))
    return column_value

In [ ]:
ALL_WORD_LIST = []
for tab in data:
    ALL_WORD_LIST.append(word_extract_value(tab))
    

In [ ]:
with open("./result_vector/table2vecFastText_ALL_WORD_LIST.pkl","rb") as f:
    ALL_WORD_LIST = pickle.load(f)

In [ ]:
ALL_WORD_LIST_DB = []
for word_list in ALL_WORD_LIST:
    ALL_WORD_LIST_DB.append([i.replace(' ','') for i in word_list])
print(ALL_WORD_LIST_DB[:2])

In [ ]:
def save_generator_to_file(generator, file_path):
    with open(file_path, 'wb') as f:
        for data in generator:
            pickle.dump(data, f)

# 제너레이터를 사용하여 데이터를 청크 단위로 저장
save_generator_to_file(ALL_WORD_LIST_DB, "./result_vector/table2vecFastText_ALL_WORD_LIST_DB.pkl")


In [ ]:
len(ALL_WORD_LIST)

In [ ]:
# 단어 리스트를 문자열로 변환
corpus = [' '.join(word_list) for word_list in ALL_WORD_LIST_DB]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

#딕셔너리 생성
ALL_TF_IDF_DICT= {}
# TF-IDF 값이 높은 상위 10개의 단어 추출
feature_names = vectorizer.get_feature_names_out()
for i, word_list in enumerate(ALL_WORD_LIST_DB):
    tfidf_values = X.toarray()[i]
    if '@' in file_list[i]:
        top_indices = tfidf_values.argsort()[-30:]
    else:
        top_indices = tfidf_values.argsort()[-10:]
    top_words = [feature_names[j] for j in top_indices]
    ALL_TF_IDF_DICT[file_list[i]] = top_words

In [ ]:
with open("./result_vector/table2vecFastText_ALL_TF_IDF_DICT.pkl","rb") as f:
    ALL_TF_IDF_DICT = pickle.load(f)

In [ ]:
def TF_IDF_OV(joined_tab):
    tab_col1,tab_col2,tab_col3 = joined_tab.split(" @ ")
    tab1,col1 = tab_col1.split("^")
    tab2,col2 = tab_col2.split("^")
    tab3,col3 = tab_col3.split("^")
    col3 = col3[:-4]
    
    sum = 0
    for word in (ALL_TF_IDF_DICT[tab1] + ALL_TF_IDF_DICT[tab2] + ALL_TF_IDF_DICT[tab3]):
        if word in ALL_TF_IDF_DICT[joined_tab]:
            sum+=1
    
    

    return sum / len(ALL_TF_IDF_DICT[joined_tab])

In [ ]:
query = ['agriculture-value-added-per-worker-wdi new.csv^country',
 '2022.csv^Country',
 '2016.csv^Country',
 'IPL Matches 2008-2020.csv^city',
 '2019_nCoV_20200121_20200128.csv^Province/State',
 'ingredients.csv^ing_id',
 'medications.csv^Disease',
 'IPL Matches 2008-2020.csv^player_of_match',
 '18_19_premier_league_Squad Possession.csv^Squad',
 'matches.csv^team2',
 'diets.csv^Disease',
 '2021.csv^Regional indicator',
 'rugs.csv^HHA Concept 1',
 '2015.csv^Country',
 'matches.csv^winner',
 '2019_world_happiness.csv^Country or region',
 'loan-train.csv^Loan_ID',
 'global_transport.csv^Country Code',
 'match_info_data.csv^player_of_match',
 'world-happiness-report-2021.csv^Regional indicator',
]

In [ ]:
#전체적으로 상관계수 계산
cos_value = []
TF_IDF_value = []
for q in query:
    for key, value in cos_dict.items():
        if key.split(" @ ")[0] == q or key.split(" @ ")[1] == q or key.split(" @ ")[2] == q:
            cos_value.append(value)
            TF_IDF_value.append(TF_IDF_OV(key))
correlation, p_value = pearsonr(cos_value, TF_IDF_value)
print(f"20개의 Query 테이블을 소스 테이블로 사용하는 모든 조인 융합 테이블의 cosine 유사도, TF-IDF-OV의 상관계수 : {correlation}")

In [ ]:
#각각 상관계수 계산
corr_list = []
for q in query:
    cos_value = []
    TF_IDF_value = []
    for key, value in cos_dict.items():
        if key.split(" @ ")[0] == q or key.split(" @ ")[1] == q or key.split(" @ ")[2] == q:
            cos_value.append(value)
            TF_IDF_value.append(TF_IDF_OV(key))
    correlation_matrix = np.corrcoef(cos_value, TF_IDF_value)
    corr_list.append(correlation_matrix[0][1])
    print(q, correlation_matrix[0][1])
# NaN 값 제거
data_without_nan = [x for x in corr_list if not np.isnan(x)]

# 평균 계산
average = sum(data_without_nan) / len(data_without_nan)
print("20개 Query 테이블 각각에 대해 상관계수를 구하고 이를 평균낸 값 :", average)


In [ ]:
for i in range(1,6):
    print(np.average(globals()['tier{}'.format(i)]))

In [ ]:
#구간 5개
for i in range(1,6):
    globals()['tier{}'.format(i)] = []
    
for i in range(10):
    print(f"\n{i+1}티어 조인\n")
    
    if i==9:
        for key, value in cos_dict.items():
            if (10-(i+1))/10 <= value <= (10-i)/10:
                try:
                    tier10.append(TF_IDF_OV(key))
                    print(key)
                except Exception as e:
                    # 그 외 모든 예외 처리
                    print("An error occurred:", e)
    else:

        for key, value in cos_dict.items():
            if (10-(i+1))/10 < value <= (10-i)/10:
                try:
                    globals()['tier{}'.format(i+1)].append(TF_IDF_OV(key))
                    print(key)
                except Exception as e:
                    # 그 외 모든 예외 처리
                    print("An error occurred:", e)